In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [25]:
column_names = [
    'area', 'perimeter', 'compactness', 'length_of_kernel',
    'width_of_kernel', 'asymmetry_coefficient', 'length_of_kernel_groove',
    'class'
]

In [34]:
data = pd.read_csv('C:/Users/ghom9/Downloads/seeds_dataset.txt', delim_whitespace=True, names=column_names)  # CSV 파일 경로
data

C:\Users\ghom9\AppData\Local\Temp\ipykernel_18936\2021779710.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv('C:/Users/ghom9/Downloads/seeds_dataset.txt', delim_whitespace=True, names=column_names)  # CSV 파일 경로


,area,perimeter,compactness,length_of_kernel,width_of_kernel,asymmetry_coefficient,length_of_kernel_groove,class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [29]:
# 입력 (X), 출력 (y) 분리
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values - 1  # 클래스가 1, 2, 3으로 되어 있어서 0부터 시작하도록 조정

In [30]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [31]:
# 원-핫 인코딩
y_categorical = to_categorical(y)

In [32]:
# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

In [36]:
# 모델 구성
model = Sequential([
    Dense(16, activation='relu', input_shape=(7,)),
    Dense(12, activation='relu'),
    Dense(3, activation='softmax')  # 클래스가 3개
])

C:\Users\ghom9\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
# 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 학습
history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=50,
                    batch_size=8,
                    verbose=1)

Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.5400 - loss: 0.9648 - val_accuracy: 0.6765 - val_loss: 0.8528
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6650 - loss: 0.9143 - val_accuracy: 0.7059 - val_loss: 0.7834
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6939 - loss: 0.8413 - val_accuracy: 0.7353 - val_loss: 0.7251
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6855 - loss: 0.7840 - val_accuracy: 0.8235 - val_loss: 0.6692
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7453 - loss: 0.7320 - val_accuracy: 0.9118 - val_loss: 0.6122
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7133 - loss: 0.7168 - val_accuracy: 0.9118 - val_loss: 0.5575
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7972 - loss: 0.5890 - val_accuracy: 0.9118 - val_loss: 0.5021
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8182 - loss: 0.5497 - val_accuracy: 0.9118 - va

In [38]:
# 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.8948 - loss: 0.1875
Test Accuracy: 0.9048


In [39]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# 예측: 확률 → 클래스 인덱스
y_pred_probs = model.predict(X_test)
y_pred_class = np.argmax(y_pred_probs, axis=1)

# 실제: 원-핫 → 클래스 인덱스
y_test_class = np.argmax(y_test, axis=1)

# 평가 결과 출력
print("Classification Report:")
print(classification_report(y_test_class, y_pred_class, digits=4))

print("Confusion Matrix:")
print(confusion_matrix(y_test_class, y_pred_class))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Classification Report:
              precision    recall  f1-score   support

           0     0.8182    0.8182    0.8182        11
           1     1.0000    1.0000    1.0000        14
           2     0.8824    0.8824    0.8824        17

    accuracy                         0.9048        42
   macro avg     0.9002    0.9002    0.9002        42
weighted avg     0.9048    0.9048    0.9048        42

Confusion Matrix:
[[ 9  0  2]
 [ 0 14  0]
 [ 2  0 15]]
